## Requirements

In [ ]:
%pip install youtube-transcript-api openai pyonepassword sqlite_utils anthropic

## Youtube transcript summarizer

In [3]:
video_id = "JP2Rwz63xeU" # ID from video URL

In [21]:
from youtube_transcript_api import YouTubeTranscriptApi
import openai
from pyonepassword import OP

op = OP()
key = op.item_get('s576pdaoabtgsza6hh4tyhfxzq').credential
client = openai.OpenAI(api_key=key)

model = "o4-mini-2025-04-16"

transcript = YouTubeTranscriptApi.get_transcript(video_id)
full_transcript = " ".join(chunk["text"] for chunk in transcript)

system_prompt = """
You are a helpful assistant.  When you answer, follow this exact format:

### Thinking:
<your private, step-by-step reasoning chain-of-thought goes here>

### Response:
<the concise summary / trader bias / personal update>
"""

user_prompt = (
    "Summarize the transcript. If the video is about trading, summarize the trader's directional bias. "
    "Also read between the lines to determine what's ACTUALLY being said and explain. "
    "Use markdown formatting. Don't skip any asset pairs mentioned. "
    "Write a personal update about the trader at the end if he mentions anything.\n\n"
    f"Transcript: {full_transcript}"
)

response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "system",  "content": system_prompt},
        {"role": "user",    "content": user_prompt},
    ],
)

# 5) Pull out the single content blob
full_content = response.choices[0].message.content

# 6) Split it on the headings and display neatly
if "### Response:" in full_content:
    thinking, response = full_content.split("### Response:", 1)
    display(Markdown(thinking.strip()))
    display(Markdown("### Response:\r\r" + response.strip()))
else:
    # fallback if the model didn't follow the exact headings
    display(Markdown(full_content))

### Thinking:
The user wants a concise summary of the long trading livestream transcript, including all asset pairs mentioned, the trader’s directional bias, and “reading between the lines” commentary. They also want a personal update about the trader if mentioned. I will parse the transcript into major themes: macro (Buffett in yen bonds, rates, safe havens, trade war), FX/indices/crypto technical and flow analysis, equity names, bond yields, strategy advice. Then I’ll capture the trader’s net directional bias: bullish JPY and EUR, bearish USD/non-US safe havens, cautious US equities, conditional on bond/gold flows, cautious Bitcoin until clear breakout. For “read between the lines,” I’ll highlight the critique of policy inconsistency, insider signals (Buffett), and carry-trade dynamics. Finally, note any personal update: the trader is coding a bot and has adjusted the livestream time.

### Response:**Summary of Key Points**  
- **Warren Buffett & Japan**  
  - Buffett’s Berkshire issued ¥90 billion in yen bonds at ~9.14%—far above euro (4.11%) or Fed rates (4–4.5%).  
  - Implies expectation of BOJ rate hikes; classic carry-trade setup.  
  - “Read between the lines”: insiders (Buffett) see big yield and currency recovery ahead.

- **Macroeconomic & Trade-War Outlook**  
  - US durable goods data mixed; labor market still “strong.”  
  - Trump’s ongoing tariff talk/frustrating flip-flops create uncertainty.  
  - Safe-haven flows into JPY, EUR, CHF, gold as investors sit on sidelines.

- **FX Technical Bias**  
  - EUR/USD: selling off, testing daily opens.  
  - USD broadly basing but likely topping if rates plateau.  
  - USD/JPY: warning against long USD/JPY—yen strength coming via BOJ intervention and rate hikes.

- **Equities & Indices**  
  - NASDAQ modest bounce but “fishy” near Asian session high; needs strong closes.  
  - Tesla flat; Nvidia volatile but trending lower toward $90 area.  
  - Small breakout names (NatWest, StoneX, Stride) lack “staple” volume—weak breadth.

- **Bond Market**  
  - 10-year yields down (~0.46% drop), 2- and 5-year yields gradually rising.  
  - Trader wants a “bond crash” to reset and fuel next rally.

- **Crypto & Bitcoin**  
  - Bitcoin rangebound: key support ~₿90 479, resistance ~₿95 000.  
  - ETF flows ~$93 m in, but AUM stuck ~573 K—no breakout fuel yet.  
  - Order-flow: bids under price, drying offers—cautious neutral bias until clean break.  
  - Altcoins (Solana) move faster; 5/13 EMA cross above 50 EMA is momentum signal.

- **“Read Between the Lines” Insights**  
  - Policy inconsistency (Trump, Fed) is a smokescreen; real moves driven by yield-chasing and insider positioning.  
  - Carry traders will flood high-yielding yen and euro bonds, pressuring USD assets.  
  - The real story isn’t tariffs or headlines but interest-rate differentials and currency dynamics.

**Trader’s Directional Bias**  
- Bullish JPY and EUR (carry-trade yields).  
- Bearish USD (expected to weaken vs. JPY/EUR).  
- Cautious US equities (pending bond-market crash/reset).  
- Neutral/borderline bearish Bitcoin until >₿95 000 breakout; bearish if <₿90 479.  
- Long gold, CHF as safe havens.

**Personal Update**  
- Trader is coding a trading bot for personal use (not to sell).  
- Adjusted today’s live-stream start time as an experiment.  
- Sending “mad love” and heading to another session later tonight.

In [18]:
response.choices[0].message.content

''

In [15]:
full_transcript

'so what\'s Warren Buffett up to now we\'ve been paying attention to what this dude has been getting on with Japan and it might not be a good thing ladies and gentlemen maybe he\'s getting himself ready for a specific position that could actually come into fruition based on what Japan has in store for us welcome to today\'s live stream ladies and gentlemen we\'re going to try and keep this one again nice and short and sweet give you a few things that you need to be aware of on what is actually developing across the board we\'re going to dive into what\'s happening with Forex Bitcoin and all of that madness and fun and games so if you are new hit the like button subscribe the market opens in roughly around 15 minutes time so we\'re going to try and get as much information out to you as possible so I hope you\'re all doing well i\'m not late i\'m just changing the time for the live that\'s that just for today just to see how it goes so let\'s get with the program ladies and gentlemen and

## Database

In [16]:
from sqlite_utils import Database
from datetime import datetime

# Initialize the database
db = Database("video_data.db")
table = "video_analysis"

# Check for duplicate video_id
if not db[table].exists() or not any(row['video_id'] == video_id for row in db[table].rows):
    # Insert a new row into the video_analysis table
    db[table].insert({
        "datetime": datetime.now(),
        "video_id": video_id,
        "transcript": full_transcript,
        "model": model,
        "ai_explanation": response.content[1].text
    })
else:
    print(f"Video ID {video_id} already exists in the database.")

AttributeError: 'ChatCompletion' object has no attribute 'content'

## Possible future work:

To enhance the morning summary for a day trader focusing on cryptocurrency, you can incorporate additional data points that provide a comprehensive view of the market. Here are some suggestions:

1. **Market Sentiment Analysis**:
   - Use sentiment analysis tools to gauge the overall market sentiment from social media platforms like Twitter, Reddit, and crypto forums.
   - Incorporate sentiment scores or trends to provide insights into the general mood of the market.

2. **Technical Indicators**:
   - Include key technical indicators such as moving averages (e.g., 50-day, 200-day), RSI (Relative Strength Index), MACD (Moving Average Convergence Divergence), and Bollinger Bands.
   - Highlight any significant technical patterns or signals (e.g., golden cross, death cross).
   - Ben Cowan (into the crypto verso): bull market support band, dominance

3. **News and Events**:
   - Summarize recent news articles or headlines that could impact the crypto market, such as regulatory changes, major partnerships, or technological advancements.
   - Include a calendar of upcoming events like earnings reports, economic data releases, or major conferences.

4. **On-Chain Data**:
   - Provide insights from on-chain data such as transaction volumes, active addresses, and large transactions (whale movements).
   - Highlight any unusual activity that could indicate market manipulation or significant interest.

5. **Volatility Index**:
   - Include a measure of market volatility, such as the Crypto Volatility Index (CVI), to help traders assess potential risk levels.

6. **Exchange Flows**:
   - Track the inflow and outflow of cryptocurrencies on major exchanges to identify potential buying or selling pressure.

7. **Correlation with Other Markets**:
   - Analyze the correlation between cryptocurrencies and other asset classes like stocks, gold, or fiat currencies to understand broader market trends.

8. **Regulatory Updates**:
   - Provide updates on any regulatory developments that could affect the crypto market, such as new laws or enforcement actions.

9. **Macro-Economic Indicators**:
   - Include relevant macroeconomic indicators such as interest rates, inflation data, and employment figures that could influence market sentiment.

10. **Community and Developer Activity**:
    - Track activity on platforms like GitHub to gauge the level of development and community engagement for specific cryptocurrencies.

By incorporating these data points, you can provide a more holistic and actionable morning summary for day traders in the crypto market.

## Experiments

In [ ]:
%pip install tradingview-ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from tradingview_ta import TA_Handler, Interval, Exchange

btc = TA_Handler(
    symbol="BTCUSDT.P",
    screener="crypto",
    exchange="binance",
    interval=Interval.INTERVAL_1_DAY
)
print("BTCUSDT Futures", btc.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

BTCUSDT Futures {'RECOMMENDATION': 'SELL', 'BUY': 3, 'SELL': 14, 'NEUTRAL': 9}


In [ ]:
im okay noy u wanna cojmme back 



SyntaxError: invalid syntax (1229904217.py, line 1)